<a href="https://colab.research.google.com/github/mishafoniakov/SberTalentCase_Horizont/blob/main/SberTalentCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Talent Case Contest 2023**

---





# 0. Подготовка рабочей среды

## 0.1. Скачивание нужных модулей

In [ ]:
!pip install pandas
!pip install numpy
!pip install torch
!pip install nltk
!pip install gensim
!pip install word2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=141242 sha256=5bd31e831af5bf690e58c6243efc9bc716d7397231253633f78e0db96fa4c14f
  Stored in directory: /root/.cache/pip/wheels/6a/fa/d1/e03e8c10e0e2aa5c7b6e2b46b4a1c715d140283853937bb4b1
Successfully built word2vec


## 0.2. Запуск датасета

In [ ]:
import pandas as pd

sample = pd.read_json('sample.json', encoding='utf-8')
print(sample.shape)
sample.head(5)

(412, 2)


,id,text
0,1,Ты нашёл их или нет?
1,2,Почему она так со мной поступает?
2,3,Никто туда больше не ходит.
3,4,У него с собой не было тогда денег.
4,5,Почему они с нами так поступают?


# 1. Предобрадотка данных

In [ ]:
import numpy as np
import re
import torch
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import gensim.models
from pymystem3 import Mystem

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 1.1. Очистка первичных данных

Необходимо проверить предложения на их типы: повествовательное, восклицательное и вопросительное. Это определяет последний знак препинания в предложении. Если предложение оканчивается на ".", то оно повествовательное, если на "!" - восклицательное, если на "?", то вопросительное.

In [ ]:
sample['type'] = sample['text'].apply(lambda x: x[-1])

Затем необходимо будет очистить предложения и привести их к общему виду:

> Удалить лишние пробелы

> Убрать прописные буквы: сделать все буквы строчные

> Убрать все символы кроме цифр и букв

> Провести лемматизацию слов в предложении: привести все слова к начальной форме

> Также дополнительно провести очистку, оставив только цифры и буквы: возможно такие остались после лемматизации

> Также повторно удалить возможные возникшие дополнительные пробелы

> Очистить предложения от стоп-слов

> Провести токенизацию получившихся предложений

In [ ]:
nltk.download('stopwords')
STOPWORDS = list(set(stopwords.words('russian')))
m = Mystem()

def text_prepare(txt):
    txt = re.sub('\s+', ' ', txt)
    txt = txt.replace('\\', '').lower().strip()
    txt = re.sub('[^0-9а-яА-ЯЁё ]+', '', txt)
    txt = ''.join(m.lemmatize(txt))
    txt = re.sub('[^0-9а-яА-ЯЁё ]+', '', txt)
    txt = re.sub('\s+', ' ', txt)
    return txt

def filter_stopwords(tokens):
    return [w for w in tokens if not w.lower() in STOPWORDS]

def tokenize(text):
    tokens = text.split()
    tokens = filter_stopwords(tokens)
    return tokens

sample['cleaned_text'] = sample['text'].apply(text_prepare)
sample['cleaned_text'] = sample['cleaned_text'].apply(tokenize)
sample.head(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


,id,text,type,cleaned_text
0,1,Ты нашёл их или нет?,?,[находить]
1,2,Почему она так со мной поступает?,?,"[почему, поступать]"
2,3,Никто туда больше не ходит.,.,"[никто, туда, ходить]"
3,4,У него с собой не было тогда денег.,.,[деньги]
4,5,Почему они с нами так поступают?,?,"[почему, поступать]"


## 1.2. Формирование датасета для анализа

Далее необходимо получить cartesian product датасета: сделать сочетание предложений каждое с каждым и переименовать получившиеся колонки для более удобного чтения

> Делаем cross join нашего датасета

> Убираем повторяющиеся парные значения

> Выбираем интересующие нас столбцы

> Необходимо убрать пары предложений с разными типами, поскольку они не могут быть рерайтами

> Переименовываем столбцы для более удобного чтения

In [ ]:
sample_cross = sample.merge(sample, how = 'cross')
sample_cross = sample_cross.loc[(sample_cross['id_x'] != sample_cross['id_y'])]
sample_cross = sample_cross[['text_x', 'text_y', 'type_x', 'type_y', 'cleaned_text_x', 'cleaned_text_y']]
sample_cross = sample_cross.rename(columns={'text_x': 'init_text', 'text_y': 'cand_text', 'type_x': 'init_type', 'type_y': 'cand_type', 'cleaned_text_x': 'init_tokens', 'cleaned_text_y': 'cand_tokens'})
sample_cross = sample_cross.loc[sample_cross['init_type'] == sample_cross['cand_type']]
sample_cross.head(5)

,init_text,cand_text,init_type,cand_type,init_tokens,cand_tokens
1,Ты нашёл их или нет?,Почему она так со мной поступает?,?,?,[находить],"[почему, поступать]"
4,Ты нашёл их или нет?,Почему они с нами так поступают?,?,?,[находить],"[почему, поступать]"
8,Ты нашёл их или нет?,Что сделал Том с деньгами?,?,?,[находить],"[сделать, деньги]"
9,Ты нашёл их или нет?,Том меня сейчас хочет видеть?,?,?,[находить],"[хотеть, видеть]"
11,Ты нашёл их или нет?,Тебе это всё нравится?,?,?,[находить],"[это, нравиться]"


# 2. Создание модели Word2Vec и алгоритма

In [ ]:
from gensim.models.word2vec import Word2Vec

##2.1. Создание модели Word2Vec

Создаём модель Word2Vec c минимальным значением токенизированного предложения 1

In [ ]:
model = gensim.models.Word2Vec(sample['cleaned_text'], min_count=1)

## 2.2. Создание алгоритма для поиска рерайтов

Алгоритм предлагаем следующий:

> Из полученного датасета выводим пару токенизированных предложений

> Внутри этой пары сравниваем попарно слова из одного и другого токенизированного предложения с присваиванием каждому слову вектор

> Считаем cosine similarity для такой пары

> Выводим средний cosine similarity из каждой пары токенизированных предложений

> Если средний cosine similarity больше или равен 0.5 - то это рерайт

In [ ]:
n = sample_cross.shape[0]
init = sample_cross['init_tokens'].values
cand = sample_cross['cand_tokens'].values
coeff = [0] * n
for i in range(n):
    m, l = len(init[i]), len(cand[i])
    cossim_counter = 0
    for j in range(m):
        for k in range(l):
            emb_1 = torch.FloatTensor(model.wv[init[i][j]]).unsqueeze(0)
            emb_2 = torch.FloatTensor(model.wv[cand[i][k]]).unsqueeze(0)
            cossim = abs((torch.nn.functional.cosine_similarity(emb_1, emb_2)).item())
            cossim_counter += cossim
    if m * l == 0:
        coeff[i] = 0
    else:
        coeff[i] = cossim_counter / (m * l)
sample_cross['coeff'] = coeff

## 2.3. Результат

Выводим получившийся результат и сохраняем его в файл JSON

In [ ]:
algorithm_1 = sample_cross.loc[sample_cross['coeff'] >= 0.5]
algorithm_1 = algorithm_1[['init_text', 'cand_text', 'coeff']]
print(f'Выявлено {algorithm_1.shape[0]} пар строк рерайта')
algorithm_1.head(20)

Выявлено 422 пар строк рерайта


,init_text,cand_text,coeff
416,Почему она так со мной поступает?,Почему они с нами так поступают?,0.518013
426,Почему она так со мной поступает?,Почему она так с ней поступает?,0.518013
481,Почему она так со мной поступает?,Почему они так с ним поступают?,0.518013
561,Почему она так со мной поступает?,Почему он так со мной поступает?,0.518013
583,Почему она так со мной поступает?,Почему она с ним так поступает?,0.518013
645,Почему она так со мной поступает?,Почему она так с ним поступает?,0.518013
1248,У него с собой не было тогда денег.,У него тогда не было с собой денег.,1.000000
1466,У него с собой не было тогда денег.,Им были нужны деньги.,0.508259
1516,У него с собой не было тогда денег.,У меня сейчас есть немного денег.,0.540911
1565,У него с собой не было тогда денег.,Им нужны были деньги.,0.508259


In [ ]:
algorithm_1.to_json('word2vec.json')

# 3. Создание модели KeyedVectors и алгоритма

In [ ]:
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile

## 3.1. Формирование модели Word2Vec с помощью модуля **KeyedVectors**

Создаём модель KeyedVectors для построения векторов

In [ ]:
kmodel = gensim.models.Word2Vec(sample['cleaned_text'], min_count=1)
word_vectors = kmodel.wv
fname = get_tmpfile('vectors.kv')
word_vectors.save(fname)
word_vectors = KeyedVectors.load(fname, mmap='r')

## 3.2. Создание алгоритма для поиска рерайтов

Алгоритм предлагается следующий:

> Из полученного датасета из каждого предложения каждой пары берём поэлементное среднее и объединяем средние значения

> Получаем числовой вектор из каждого предложения

> Считаем cosine similarity для каждой пары предложений

> Если средний cosine similarity больше или равен 0.6 - то это рерайт

In [ ]:
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = word2vec.vectors.shape[1]

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec.key_to_index]
                or [np.zeros(self.dim)], axis=0)
        for words in X
        ])

vectorizer = EmbeddingVectorizer(word_vectors)

n = sample_cross.shape[0]
init = sample_cross['init_tokens'].values
cand = sample_cross['cand_tokens'].values
coeff = [0] * n
for i in range(n):
    emb_init = torch.FloatTensor(vectorizer.transform([init[i]]))
    emb_cand = torch.FloatTensor(vectorizer.transform([cand[i]]))
    coeff[i] = abs(torch.nn.functional.cosine_similarity(emb_init, emb_cand).item())
sample_cross['coeff'] = coeff

##3.3. Результаты

Выводим получившийся результат и сохраняем его в файл JSON

In [ ]:
algorithm_2 = sample_cross.loc[(sample_cross['init_type'] == sample_cross['cand_type']) & (sample_cross['coeff'] >= 0.6)]
algorithm_2 = algorithm_2[['init_text', 'cand_text', 'coeff']]
print(f'Выявлено {algorithm_2.shape[0]} пар строк рерайта')
algorithm_2.head(20)

Выявлено 754 пар строк рерайта


,init_text,cand_text,coeff
416,Почему она так со мной поступает?,Почему они с нами так поступают?,1.000000
426,Почему она так со мной поступает?,Почему она так с ней поступает?,1.000000
481,Почему она так со мной поступает?,Почему они так с ним поступают?,1.000000
561,Почему она так со мной поступает?,Почему он так со мной поступает?,1.000000
583,Почему она так со мной поступает?,Почему она с ним так поступает?,1.000000
645,Почему она так со мной поступает?,Почему она так с ним поступает?,1.000000
1089,Никто туда больше не ходит.,Никто больше туда не ходит.,1.000000
1248,У него с собой не было тогда денег.,У него тогда не было с собой денег.,1.000000
1286,У него с собой не было тогда денег.,Мне никогда не нужно было столько денег.,0.635866
1466,У него с собой не было тогда денег.,Им были нужны деньги.,0.707090


In [ ]:
algorithm_2.to_json('keyedvectors.json')

# 4. Создание алгоритма на основе модели word2vec с использованием токенов и биграммов

Алгоритм предлагается следующий:

> Для каждого предложения выводим биграммы. Таким образом, в нашем пайплайне есть две колонки: колонка с токенами и колонка с биграммами

> Для каждого биграмма: внутри биграмма для каждого токена находим word2vec вектор. Берём минимальное значение из полученного вектора и получаем коэффициент

> Для каждого токена: находим минимальное значение для каждого токена. Берём минимальное значение из полученного вектора и получаем коэффициент

> Сравниваем коэффициенты биграммов. Если они равны и при этом не равны 0 - то это копирайт

> Если коэффициент у биграмма нулевой - сравниваем коэффициенты токенов. Если коэффициенты у токенов равны - то это копирайт

## 4.1. Формирование модели Word2Vec и алгоритма для поиска рерайтов

In [ ]:
from nltk import ngrams

model = gensim.models.Word2Vec(sample['cleaned_text'], min_count=1)

def coeff_build(text, type_='ngram'):
    if len(text) == 0:
        return 0
    elif type_ == 'tokens':
        return np.mean([model.wv[token] for token in text])
    else:
        return np.mean([[model.wv[word] for word in ngram] for ngram in text])

sample_cross["init_ngrams"] = sample_cross["init_tokens"].apply(lambda x: list(ngrams(x, 2)))
sample_cross["cand_ngrams"] = sample_cross["cand_tokens"].apply(lambda x: list(ngrams(x, 2)))
sample_cross['init_ngram_coeff'] = sample_cross['init_ngrams'].apply(lambda x: coeff_build(x, type_='ngram'))
sample_cross['cand_ngram_coeff'] = sample_cross['cand_ngrams'].apply(lambda x: coeff_build(x, type_='ngram'))
sample_cross['init_token_coeff'] = sample_cross['init_tokens'].apply(lambda x: coeff_build(x, type_='tokens'))
sample_cross['cand_token_coeff'] = sample_cross['cand_tokens'].apply(lambda x: coeff_build(x, type_='tokens'))

## 4.2. Получение результатов

In [ ]:
algorithm_3 = sample_cross.loc[((sample_cross['init_ngram_coeff'] == sample_cross['cand_ngram_coeff']) &
 (sample_cross['init_ngram_coeff'] != 0)) | ((sample_cross['init_token_coeff'] == sample_cross['cand_token_coeff']) &
 (sample_cross['init_token_coeff'] != 0))]

algorithm_3 = algorithm_3[['init_text', 'cand_text', 'init_ngram_coeff', 'cand_ngram_coeff', 'init_token_coeff', 'cand_token_coeff']]
print(f'Выявлено {algorithm_3.shape[0]} пар строк рерайта')
algorithm_3.head(20)

Выявлено 344 пар строк рерайта


,init_text,cand_text,init_ngram_coeff,cand_ngram_coeff,init_token_coeff,cand_token_coeff
416,Почему она так со мной поступает?,Почему они с нами так поступают?,-0.000077,-0.000077,-0.000077,-0.000077
426,Почему она так со мной поступает?,Почему она так с ней поступает?,-0.000077,-0.000077,-0.000077,-0.000077
481,Почему она так со мной поступает?,Почему они так с ним поступают?,-0.000077,-0.000077,-0.000077,-0.000077
561,Почему она так со мной поступает?,Почему он так со мной поступает?,-0.000077,-0.000077,-0.000077,-0.000077
583,Почему она так со мной поступает?,Почему она с ним так поступает?,-0.000077,-0.000077,-0.000077,-0.000077
645,Почему она так со мной поступает?,Почему она так с ним поступает?,-0.000077,-0.000077,-0.000077,-0.000077
1089,Никто туда больше не ходит.,Никто больше туда не ходит.,-0.000522,-0.000522,-0.000725,-0.000725
1248,У него с собой не было тогда денег.,У него тогда не было с собой денег.,0.000000,0.000000,0.001284,0.001284
1649,Почему они с нами так поступают?,Почему она так со мной поступает?,-0.000077,-0.000077,-0.000077,-0.000077
1662,Почему они с нами так поступают?,Почему она так с ней поступает?,-0.000077,-0.000077,-0.000077,-0.000077


In [ ]:
algorithm_3.to_json('ngrams_tokens.json')

# 5. Создание алгоритма на основе вектора BagOfWords с использованием токенов

## 5.1. Формирование модели BagOfWords и алгоритма для поиска рерайтов

Алгоритм предлагается следующий:
> Получаем список уникальных слов из всего датасета

> Создаём вектор из этого уникального списка слов:

>> изначально создаём нулевой вектор

>> длина вектора каждого предложения равна длине этого списка уникальных слов

>> если слово встречается в предложении - ставим соответствующую цифру повторов в индексе соответствующему слову в списке слов

> Cравниваем вектора каждого преждожения попарно

> Считаем схожесть вектора по cosine similarity

In [ ]:
def to_1D(series):
    return [x for _list in series for x in _list]

def BoW(series):
    tokens = to_1D(series)
    voc = list(set(tokens))
    n_tokens = len(voc)
    vector = [0] * n_tokens
    vector_list = []
    count = 0
    for each_doc in series:
        for i in voc:
            for word in each_doc:
                if i == word:
                    count += 1
            vector[voc.index(i)] = count
            count = 0
        vector_list.append(vector)
        vector = ([0] * n_tokens)
    return vector_list

sample['BoW_tokens'] = BoW(sample['cleaned_text'])

## 5.2. Получение результатов

In [ ]:
sample_cross = sample.merge(sample, how = 'cross')
sample_cross = sample_cross.loc[(sample_cross['id_x'] != sample_cross['id_y'])]
sample_cross = sample_cross[['text_x', 'text_y', 'type_x', 'type_y', 'BoW_tokens_x', 'BoW_tokens_y']]
sample_cross = sample_cross.rename(columns={'text_x': 'init_text', 'text_y': 'cand_text', 'type_x': 'init_type', 'type_y': 'cand_type', 'BoW_tokens_x': 'init_bow', 'BoW_tokens_y': 'cand_bow'})
sample_cross = sample_cross.loc[sample_cross['init_type'] == sample_cross['cand_type']]
algorithm_4 = sample_cross.loc[sample_cross['init_bow'] == sample_cross['cand_bow']]
print(f'Выявлено {algorithm_4.shape[0]} пар строк рерайта')
algorithm_4[['init_text', 'cand_text', 'init_bow', 'cand_bow']].head(20)

Выявлено 386 пар строк рерайта


,init_text,cand_text,init_bow,cand_bow
416,Почему она так со мной поступает?,Почему они с нами так поступают?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
426,Почему она так со мной поступает?,Почему она так с ней поступает?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
481,Почему она так со мной поступает?,Почему они так с ним поступают?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
561,Почему она так со мной поступает?,Почему он так со мной поступает?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
583,Почему она так со мной поступает?,Почему она с ним так поступает?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
645,Почему она так со мной поступает?,Почему она так с ним поступает?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1089,Никто туда больше не ходит.,Никто больше туда не ходит.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1248,У него с собой не было тогда денег.,У него тогда не было с собой денег.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1649,Почему они с нами так поступают?,Почему она так со мной поступает?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1662,Почему они с нами так поступают?,Почему она так с ней поступает?,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
algorithm_4.to_json('bow.json')

# 6. Создание алгоритма на основе вектора KeyeydVectors с кластеризацией

In [ ]:
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile

Алгоритм предлагается следующий:
> Создаём KeyedVector из каждого предложения: из полученного датасета из каждого предложения каждой пары берём поэлементное среднее и объединяем средние значения и получаем числовой вектор из каждого предложения

> Создаём Bag of Words из каждого предложения: изначально создаём нулевой вектор, длина вектора каждого предложения равна длине этого списка уникальных слов, если слово встречается в предложении - ставим соответствующую цифру повторов в индексе соответствующему слову в списке слов, получаем числовой вектор из каждого предложения

> Распределяем вектора по 300 кластерам

> Сопоставляем пары номеров кластеров по KeyedVectors и BagOfWords

> Если по одному или по другому вектору кластера равны - это копирайт

## 6.1. Формирование модели Word2Vec с помощью модуля **KeyedVectors** и создание алгоритма для получения векторов

In [ ]:
vectorizer = EmbeddingVectorizer(word_vectors)

n = sample.shape[0]
tokens = sample['cleaned_text'].values
vector = ['x'] * n
for i in range(n):
    emb = vectorizer.transform([tokens[i]]).squeeze()
    vector[i] = emb
sample['vector'] = vector
vector = np.array([list(x) for x in sample['vector']])

sample['bow'] = BoW(sample['cleaned_text'])
bow = np.array([list(x) for x in sample['bow']])

## 6.2. Создание кода для формирования кластеров

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=300, random_state=0, n_init="auto")
sample['kv_clusters'] = kmeans.fit_predict(vector)
sample['bow_clusters'] = kmeans.fit_predict(bow)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (289) found smaller than n_clusters (300). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (282) found smaller than n_clusters (300). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_


## 6.3. Получение результатов

In [ ]:
sample_cross = sample.merge(sample, how = 'cross')
sample_cross = sample_cross.loc[(sample_cross['id_x'] != sample_cross['id_y'])]
sample_cross = sample_cross[['text_x', 'text_y', 'type_x', 'type_y', 'kv_clusters_x', 'kv_clusters_y', 'bow_clusters_x', 'bow_clusters_y']]
sample_cross = sample_cross.rename(columns={'text_x': 'init_text', 'text_y': 'cand_text', 'type_x': 'init_type', 'type_y': 'cand_type', 'kv_clusters_x': 'init_kv_cluster', 'kv_clusters_y': 'cand_kv_cluster', 'bow_clusters_x': 'init_bow_cluster', 'bow_clusters_y': 'cand_bow_cluster'})
sample_cross = sample_cross.loc[sample_cross['init_type'] == sample_cross['cand_type']]
algorithm_5 = sample_cross.loc[(sample_cross['init_kv_cluster'] == sample_cross['cand_kv_cluster']) |
 (sample_cross['init_bow_cluster'] == sample_cross['cand_bow_cluster'])]
print(f'Выявлено {algorithm_5.shape[0]} пар строк рерайта')
algorithm_5[['init_text', 'cand_text', 'init_kv_cluster', 'cand_kv_cluster', 'init_bow_cluster', 'cand_bow_cluster']].head(20)

Выявлено 386 пар строк рерайта


,init_text,cand_text,init_kv_cluster,cand_kv_cluster,init_bow_cluster,cand_bow_cluster
416,Почему она так со мной поступает?,Почему они с нами так поступают?,14,14,14,14
426,Почему она так со мной поступает?,Почему она так с ней поступает?,14,14,14,14
481,Почему она так со мной поступает?,Почему они так с ним поступают?,14,14,14,14
561,Почему она так со мной поступает?,Почему он так со мной поступает?,14,14,14,14
583,Почему она так со мной поступает?,Почему она с ним так поступает?,14,14,14,14
645,Почему она так со мной поступает?,Почему она так с ним поступает?,14,14,14,14
1089,Никто туда больше не ходит.,Никто больше туда не ходит.,88,88,42,42
1248,У него с собой не было тогда денег.,У него тогда не было с собой денег.,13,13,54,54
1649,Почему они с нами так поступают?,Почему она так со мной поступает?,14,14,14,14
1662,Почему они с нами так поступают?,Почему она так с ней поступает?,14,14,14,14


In [ ]:
algorithm_5.to_json('clusters.json')